## Extra works
- Use PostgreSQL, to insert data in netflix_titles.csv
- Include embeddings into PostgreSQL

In [13]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [14]:

import psycopg2

# Connect to PostgreSQL (update credentials as needed)
conn = psycopg2.connect(
    dbname="netflix_db",
    user=os.getenv("POSTGRES_USERNAME_DB", "postgres"),
    password=os.getenv("POSTGRES_PASSWORD_DB", "Post!234"),
    host=os.getenv("POSTGRES_DATABASE_HOST", "localhost"),
    port=os.getenv("POSTGRES_DATABASE_PORT", "5432")
)
cur = conn.cursor()

# Create table for netflix_titles
cur.execute("""
CREATE TABLE IF NOT EXISTS netflix_titles (
    show_id TEXT PRIMARY KEY,
    type TEXT,
    title TEXT,
    director TEXT,
    "cast" TEXT,
    country TEXT,
    date_added TEXT,
    release_year INT,
    rating TEXT,
    duration TEXT,
    listed_in TEXT,
    description TEXT
)
""")
conn.commit()

cur.close()
conn.close()

In [15]:
import pandas as pd
import psycopg2
# Load data from CSV
netflix_df = pd.read_csv('data/netflix_titles.csv')

# Reconnect to PostgreSQL
conn = psycopg2.connect(
    dbname="netflix_db",
    user=os.getenv("POSTGRES_USERNAME_DB", "postgre"),
    password=os.getenv("POSTGRES_PASSWORD_DB", "Post!234"),
    host=os.getenv("POSTGRES_DATABASE_HOST", "localhost"),
    port=os.getenv("POSTGRES_DATABASE_PORT", "5432")
)
cur = conn.cursor()

# Insert data into netflix_titles table
for _, row in netflix_df.iterrows():
    cur.execute("""
        INSERT INTO netflix_titles (
            show_id, type, title, director, "cast", country, date_added, 
            release_year, rating, duration, listed_in, description
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (show_id) DO NOTHING
    """, (
        row['show_id'], row['type'], row['title'], row['director'], row['cast'],
        row['country'], row['date_added'], row['release_year'], row['rating'],
        row['duration'], row['listed_in'], row['description']
    ))
conn.commit()

cur.close()
conn.close()

In [71]:
# %pip install sqlalchemy

from sqlalchemy import create_engine, text

# Create SQLAlchemy engine using the same credentials
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('POSTGRES_USERNAME_DB', 'postgres')}:{os.getenv('POSTGRES_PASSWORD_DB', 'Post!234')}@{os.getenv('POSTGRES_DATABASE_HOST', 'localhost')}:{os.getenv('POSTGRES_DATABASE_PORT', '5432')}/netflix_db"
)

# Query the netflix_titles table
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM netflix_titles LIMIT 5"))
    for row in result:
        print(row)

('s4305', 'TV Show', 'Hymn of Death', 'NaN', 'Lee Jong-suk, Shin Hye-sun, Kim Myeong-su, Park Seon-im, Kim Won-hae, Hwang Yeong-hee, Lee Ji-hoon, Go Bo-gyeol, Shin Jae-ha', 'South Korea', 'December 14, 2018', 2018, 'TV-14', '1 Season', 'International TV Shows, Korean TV Shows, Romantic TV Shows', 'During the Japanese colonial period, married playwright Kim U-jin falls in love with soprano Yun Sim-deok, one of Korea’s notable New Women.', '[0.024462145,0.009203575,-0.018876905,0.016271174,0.032449156,-0.021197053,0.051870357,0.033905055,-0.013914176,0.03153898,0.002262053,-0.04362198,-0 ... (9234 characters truncated) ... 0.008327475,-0.016136376,0.0059068445,0.05991382,-0.048062183,0.030414872,0.019131934,-0.04887194,-0.056101635,0.037168156,-0.050279275,-0.008977754]')
('s4307', 'Movie', 'ROMA', 'Alfonso Cuarón', 'Yalitza Aparicio, Marina de Tavira', 'Mexico, United States', 'December 14, 2018', 2018, 'R', '135 min', 'Dramas, Independent Movies, International Movies', "Director Alfons

## Create embeddings and insert them into netflix_titles



In [ ]:
%pip install google-generativeai

import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

result = genai.embed_content(
    model="models/embedding-001",
    # content=["What is the meaning of life?", "how to make a better life?"]
    content="What is the meaning of life?"
)

print(result.keys())
print(result["embedding"])
print(len(result["embedding"]))

Note: you may need to restart the kernel to use updated packages.
dict_keys(['embedding'])
[-0.00026206698, -0.055920187, -0.012463197, -0.020672267, 0.0076786787, 0.0024069808, 0.030989334, 0.01582611, -0.015852105, 0.011664646, -0.009116539, 0.0010538554, 0.036472403, -0.03241015, 0.008458409, -0.0033494884, 0.021979261, 0.015039315, 0.011996514, -0.03020258, -0.03397108, 0.010469224, -0.0036774632, 0.01323687, 0.007085988, -0.03823258, 0.018632846, -0.07036457, -0.022376658, 0.05355737, -0.058401376, 0.035850685, -0.087846205, -0.01713318, 0.04601891, -0.06547601, 0.06361521, 0.003836632, -0.05621926, 0.0153549155, 0.027993219, -0.008997662, -0.07307445, -0.029412353, -0.007533671, 0.005410707, -0.008447231, 0.03168135, 0.0041648657, -0.10778426, 0.016246738, 0.034010187, 0.080236025, -0.007318589, -6.490005e-05, -0.007139853, 0.048286457, 0.015510907, -0.029625922, -0.0059952387, 0.008322641, 0.006501486, -0.025500132, 0.063725494, -0.019506635, -0.0068314015, 0.026189998, 0.022515

In [40]:

from tenacity import (retry, stop_after_attempt, wait_random_exponential)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def embed(input):
    result = genai.embed_content(
        model="models/embedding-001",
        content=input
    )
    return result["embedding"]

In [42]:
embedded_content = embed(["What is the meaning of life?", "how to make a better life?"])
embedded_content

[[-0.00026206698,
  -0.055920187,
  -0.012463197,
  -0.020672267,
  0.0076786787,
  0.0024069808,
  0.030989334,
  0.01582611,
  -0.015852105,
  0.011664646,
  -0.009116539,
  0.0010538554,
  0.036472403,
  -0.03241015,
  0.008458409,
  -0.0033494884,
  0.021979261,
  0.015039315,
  0.011996514,
  -0.03020258,
  -0.03397108,
  0.010469224,
  -0.0036774632,
  0.01323687,
  0.007085988,
  -0.03823258,
  0.018632846,
  -0.07036457,
  -0.022376658,
  0.05355737,
  -0.058401376,
  0.035850685,
  -0.087846205,
  -0.01713318,
  0.04601891,
  -0.06547601,
  0.06361521,
  0.003836632,
  -0.05621926,
  0.0153549155,
  0.027993219,
  -0.008997662,
  -0.07307445,
  -0.029412353,
  -0.007533671,
  0.005410707,
  -0.008447231,
  0.03168135,
  0.0041648657,
  -0.10778426,
  0.016246738,
  0.034010187,
  0.080236025,
  -0.007318589,
  -6.490005e-05,
  -0.007139853,
  0.048286457,
  0.015510907,
  -0.029625922,
  -0.0059952387,
  0.008322641,
  0.006501486,
  -0.025500132,
  0.063725494,
  -0.019506635

In [35]:
with engine.connect() as connection:
    connection.execute(text("""
        ALTER TABLE netflix_titles
        ADD COLUMN IF NOT EXISTS embedding vector(768)
    """))

In [ ]:
from tqdm import tqdm

# Prepare batch data for embedding
batch_size = 50
rows = netflix_df[['show_id', 'title', 'description']].dropna().values
with engine.connect() as connection:
    for i in tqdm(range(0, len(rows), batch_size)):
        batch = rows[i:i+batch_size]
        show_ids = batch[:, 0]
        texts = [f"{title}: {desc}" for title, desc in batch[:, 1:]]
        # Generate embeddings
        embeddings = embed(texts)

        # Insert embeddings into the database
        for show_id, embedding in zip(show_ids, embeddings):
            connection.execute(
                text("UPDATE netflix_titles SET embedding = :embedding WHERE show_id = :show_id"),
                {"embedding": embedding, "show_id": show_id}
            )
        connection.commit()


100%|██████████| 177/177 [05:17<00:00,  1.79s/it]


In [81]:
import numpy as np
query = "movie about love"
embedding = embed(query)
print(str(list(embedding)))


[0.031563964, -0.016512273, 0.024190737, -0.0134637775, 0.015838902, -0.024541842, -0.024390275, 0.030985005, 0.032450173, 0.022827886, 0.02315422, -0.029679136, 0.0009337551, -0.01792666, 0.01861898, 0.029083423, 0.017570456, -0.028796108, 0.02099581, -0.026987566, -0.010240196, 0.015458901, -0.043409925, -0.0037805059, 0.0065840837, -0.026802845, 0.031759437, -0.05899208, -0.027342275, -0.013107873, -0.06020251, -0.010211064, -0.07458132, 0.024662858, 0.036195923, -0.032470543, -0.03017124, -0.0056299465, 0.019527614, -0.0098353205, 0.013378524, -0.066564195, -0.025964413, -0.016449155, 0.03900989, -0.024699619, 0.023861753, -0.044661887, 0.023196876, -0.06459096, 0.039273594, -0.014283054, 0.03112705, 0.002078736, 0.02187108, 0.0005695914, 0.08853291, 0.031089814, -0.0758752, 0.002366492, 0.0030102017, -0.020506036, -0.006052213, 0.081886195, 0.002541665, 0.0049035014, 0.013517248, 0.023683688, 0.035567466, -0.0054859403, -0.017569862, -0.014320844, 0.06186045, 0.013343466, -1.94557

In [94]:
query = "movie about love"
embedding = embed(query)

with engine.connect() as connection:
    result = connection.execute(text("""
            SELECT show_id, title, embedding
            FROM netflix_titles
            ORDER BY embedding <=> :embedding
            LIMIT 5
        """), 
        {"embedding": str(list(embedding))}
    ).fetchall()

    for row in result:
        print(f"{row[0]}, {row[1]}, {row[2]}...")  # Print first 10 elements of the embedding
    
    

s2833, All About Love, [0.062065504,-0.010405871,0.0068473252,-0.0052464716,0.04204618,-0.029987494,-0.016124276,-0.010482812,0.050300255,-0.03308697,0.054780364,-0.018481268,0.031171665,-0.013478102,-0.001944607,0.01771068,0.027235536,-0.02223916,-0.004932926,-0.0045884117,0.011849652,-0.0086080665,-0.030281214,-0.039755985,-0.025096582,-0.024778508,0.032196738,-0.045786437,-0.013548752,-0.040054176,-0.06096758,-0.008351752,-0.07981308,0.0151811745,-0.0013698431,-0.040132534,0.0010283362,-0.021896485,0.0022341579,0.014369423,0.028809136,-0.036797553,0.012076571,-0.027240328,0.03477977,0.014749514,0.016144669,-0.054417916,0.022844687,-0.04937368,-0.009814526,-0.021321304,0.007857802,-0.028186692,0.03955439,0.03842089,0.01870434,0.023678133,-0.05179825,0.004751948,-0.020363739,-0.0027884392,0.014335708,0.05720982,-0.017352369,-0.0037246544,0.008802299,-0.0002968917,0.06196913,-0.01327517,-6.4594315e-05,-0.014563083,0.09111168,0.047461413,-0.009969511,-0.058963787,-0.031573106,0.05029478